In [1]:
!pip install qiskit qiskit_machine_learning
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
def ansatz_14(n_qubits, depth):

    theta = ParameterVector('θ', 8*depth)

    qc = QuantumCircuit(n_qubits)
    for j in range(depth//2):
        for i in range(n_qubits):
            qc.ry(theta[j*n_qubits*4+i], i)

        qc.crx(theta[j*n_qubits*4+4], 3, 0)
        qc.crx(theta[j*n_qubits*4+5], 2, 3)
        qc.crx(theta[j*n_qubits*4+6], 1, 2)
        qc.crx(theta[j*n_qubits*4+7], 0, 1)


        #Warstwa 2/4(8-15)(25-31)
        for i in range(n_qubits):
            qc.ry(theta[j*n_qubits*4+8 + i], i)


        qc.crx(theta[j*n_qubits*4+12], 3, 2)
        qc.crx(theta[j*n_qubits*4+13], 0, 3)
        qc.crx(theta[j*n_qubits*4+14], 1, 0)
        qc.crx(theta[j*n_qubits*4+15], 2, 1)
    return qc

ansatz_14(4,4)

In [3]:
from qiskit_machine_learning.gradients import ParamShiftEstimatorGradient
class HybridModel:
    def __init__(self, ansatz_circuit, num_qubits, input_dimension):
        self.num_qubits = num_qubits
        self.input_dimension = input_dimension

        self.feature_map = ZZFeatureMap(feature_dimension=input_dimension, reps=1)

        self.qc = QuantumCircuit(num_qubits)
        self.qc.compose(self.feature_map,qubits=range(input_dimension), inplace=True)
        self.qc.compose(ansatz_circuit, inplace=True)

        final_circuit_params = self.qc.parameters

        feature_map_names = {p.name for p in self.feature_map.parameters}
        ansatz_names = {p.name for p in ansatz_circuit.parameters}

        # Sort the final parameters into w and input
        self.final_input_params = []
        self.final_weight_params = []

        for p in final_circuit_params:
            if p.name in feature_map_names:
                self.final_input_params.append(p)
            elif p.name in ansatz_names:
                self.final_weight_params.append(p)

        # check bo cos wczesniej nie zczytywalo ansatz
        if len(self.final_weight_params) == 0:
            print("CRITICAL ERROR: No weight parameters found in the circuit!")
            print(f"Ansatz names: {ansatz_names}")
            print(f"Circuit params: {[p.name for p in final_circuit_params]}")


        observable = SparsePauliOp.from_list([("I" * (num_qubits - 1) + "Z", 1)])

        estimator = Estimator()
        # okay skapnęłam się jeszcze, że robiłam to na Estimator, który zaraz wyjdzie z użycia w Qiskit
        # więc kolejny powinien być taki ale z nim nie testowałam kodu
        # from qiskit.primitives import StatevectorEstimator
        # estimator = StatevectorEstimator()
        gradient = ParamShiftEstimatorGradient(estimator)
        self.qnn = EstimatorQNN(
            circuit=self.qc,
            observables=observable,
            input_params=self.final_input_params,
            weight_params=self.final_weight_params,
            estimator=estimator,
            gradient = gradient
        )

    def forward(self, x, weights):
        return self.qnn.forward(x, weights)

    def backward(self, x, weights):
        _, weight_grads = self.qnn.backward(x, weights)
        if weight_grads is None:
            # If it fails, return zeros to prevent the loop from crashing
            # This was messy process
            print("Warning: Gradients were None. Returning Zeros.")
            return np.zeros((x.shape[0], len(weights)))
        return weight_grads


In [4]:
my_ansatz = ansatz_14(4, 4)
qnn = HybridModel(
    ansatz_circuit=my_ansatz,
    num_qubits=5,
    input_dimension=4
)


# Inicjalizacja wag
num_weights = qnn.qnn.num_weights
rng = np.random.default_rng(seed=42)
weights = 2 * np.pi * rng.random(num_weights)
weights = weights.flatten()
# Just to be sure spłaszczam jeszcze do D1

print(f"Weights initialized. Shape: {weights.shape}")

Weights initialized. Shape: (32,)


/tmp/ipython-input-343941416.py:37: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/tmp/ipython-input-343941416.py:38: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  gradient = ParamShiftEstimatorGradient(estimator)
/tmp/ipython-input-343941416.py:39: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  self.qnn = EstimatorQNN(


In [6]:
data = pd.read_csv('data_banknote_authentication.txt', header=None)

# Podział na cechy (X) i etykiety (y)
X = data.iloc[:, 0:4].values  # 4 parametry wejściowe (float)
y = data.iloc[:, 4].values    # Binarne wyjście (0 lub 1)

scaler = MinMaxScaler(feature_range=(0, 2*np.pi))
X_scaled = scaler.fit_transform(X)

# Podział na zestaw treningowy i testowy (80% trening, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Rozmiar treningowy: {X_train.shape}, Rozmiar testowy: {X_test.shape}")

Rozmiar treningowy: (1097, 4), Rozmiar testowy: (275, 4)


In [8]:
# okay a więc pozmieniałam trochę rzeczy względem Michała kodu
# dużo jest pilnowania by pp array miała odpowiedni kształt, bo to był problem początkowo

from sklearn.utils import shuffle

class AdamOptimizer:
    def __init__(self, params_shape, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = np.zeros(params_shape)
        self.v = np.zeros(params_shape)
        self.t = 0

    def step(self, weights, grads):
        self.t += 1
        # upewnienie się, że array będzie 1D
        # czy będzie działać bez? atm nie jestem w stanie powiedzieć
        # ale robiłam już z tym buforem bezpieczeństwa by mi nie wywaliło nagle
        # jak ktos ma wolne 8h have fun
        grads = np.asarray(grads).flatten()
        weights = np.asarray(weights).flatten()

        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grads ** 2)

        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        updated_weights = weights - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        return updated_weights

optimizer = AdamOptimizer(weights.shape, lr=0.02)
print("Optimizer Class:", type(optimizer))


EPOCHS = 20
BATCH_SIZE = 32
loss_history = []

print(f"Starting training with {num_weights} parameters...")

for epoch in range(EPOCHS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=epoch)

    epoch_loss = 0.0
    batches_count = 0

    for i in range(0, len(X_train), BATCH_SIZE):
        X_batch = X_train_shuffled[i:i + BATCH_SIZE]
        y_batch = y_train_shuffled[i:i + BATCH_SIZE]

        # 1. Forward & Backward, teraz bez tego _, przed grads, bo dopasowałam już
        # rzeczy by nie było problemu

        pred = qnn.forward(X_batch, weights)
        grads = qnn.backward(X_batch, weights)

        if grads is None or np.all(grads == 0):
             continue

        # JUST TO BE SURE, teraz obydwie są reshaped
        pred = pred.reshape(-1, 1)
        y_batch = y_batch.reshape(-1, 1)

        diff = pred - y_batch             # Shape: (32, 1)

        grad_modifier = 2 * diff          # Shape: (32, 1)

        # Multiply: (32, 1) * (32, 20), Result: (32, 20)
        weighted_grads = grad_modifier * grads

        # Collapse the 32 rows into 1 row,  Result: (20,)
        batch_grads = np.mean(weighted_grads, axis=0)

        # 4. FINAL GUARD RAIL
        # If batch_grads is somehow still wrong, force it to match weights
        if batch_grads.shape != weights.shape:
            # If we got (32, 20), this line forces it to (20,)
            # This handles the edge case where np.mean didn't reduce correctly
            # yeah a więd widzicie dużo zabezpieczeń?
            batch_grads = np.mean(batch_grads.reshape(-1, num_weights), axis=0)

        weights = optimizer.step(weights, batch_grads)

        loss = np.mean(diff ** 2)
        epoch_loss += loss
        batches_count += 1

    avg_loss = epoch_loss / batches_count
    loss_history.append(avg_loss)
    print(f"Epoch {epoch+1}/{EPOCHS} | Avg loss: {avg_loss:.4f}")

Optimizer Class: <class '__main__.AdamOptimizer'>
Starting training with 32 parameters...


KeyboardInterrupt: 

In [17]:
# Mini batch by zobaczyć co wypluł model dla pierwszych 20
test_sample = X_test[:20]
raw_output = qnn.forward(test_sample, weights)

print("Raw Model Outputs:")
print(raw_output)


Raw Model Outputs:
[[0.38387199]
 [0.34303683]
 [0.46819914]
 [0.3839113 ]
 [0.14688022]
 [0.21090933]
 [0.48669335]
 [0.17685369]
 [0.43205557]
 [0.48723074]
 [0.15230484]
 [0.73294738]
 [0.46496263]
 [0.80796705]
 [0.36395971]
 [0.41813981]
 [0.76741262]
 [0.75863253]
 [0.03196007]
 [0.75344335]]


In [23]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

subset_X = X_test[:200]
subset_y = y_test[:200]

raw_output = qnn.forward(subset_X, weights)
raw_numpy = np.array(raw_output)

# tu mam decyzje na 0.5
predictions = np.where(raw_numpy > 0.5, 1, 0).flatten()

cm = confusion_matrix(subset_y, predictions)
print("Confusion Matrix:")
print(cm)

# WNIOSKI: w zerach jest świetny, ale z jedynkami ma duży problem.
# moim zdaniem to wynikało z tego że encoding miałam zrobiony na 0,1 a nie 0, 2pi

Confusion Matrix:
[[100   9]
 [ 34  57]]
